In [1]:
!pip install pybullet imageio imageio-ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 7.5 MB/s eta 0:00:00


In [2]:
import pybullet as p
import pybullet_data
import numpy as np
import imageio
import os
import shutil
import math

# 保存フォルダ準備
if os.path.exists("frames"):
    shutil.rmtree("frames")
os.makedirs("frames", exist_ok=True)

# PyBullet初期化（GUIなし）
client = p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.resetSimulation()
p.setGravity(0, 0, -9.8)

# 地面とLaikagoロボットを読み込み
p.loadURDF("plane.urdf")
laikago_path = "laikago/laikago_toes.urdf"
robot = p.loadURDF(laikago_path, basePosition=[0, 0, 0.45],baseOrientation=p.getQuaternionFromEuler([np.pi/2, 0, -np.pi/2]))

# 初期関節設定（ただの繰り返し動作）
num_joints = p.getNumJoints(robot)
print("Number of joints:", num_joints)

# 適当なループで関節にサイン波コントロール
frames = []
for t in range(240):
    for j in range(num_joints):
        angle = 0.3 * np.sin(0.1 * t + j)
        p.setJointMotorControl2(
            robot, j, p.POSITION_CONTROL, targetPosition=angle, force=20
        )
    p.stepSimulation()

    # カメラで画像取得（必要なら位置調整）
    width, height, rgb, _, _ = p.getCameraImage(320, 240)
    rgb_img = np.reshape(rgb, (height, width, 4))[:, :, :3]
    imageio.imwrite(f"frames/frame_{t:04d}.png", rgb_img)

p.disconnect()


Number of joints: 16


In [3]:
# MP4形式で保存
video_path = "laikago_walk.mp4"
imageio.mimsave(video_path, [imageio.imread(f"frames/frame_{t:04d}.png") for t in range(240)], fps=30)

/tmp/ipython-input-3-709665643.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imageio.mimsave(video_path, [imageio.imread(f"frames/frame_{t:04d}.png") for t in range(240)], fps=30)


In [4]:
!pip install stable-baselines3 gym

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [5]:
import gym
from gym import spaces
import numpy as np
import pybullet as p
import pybullet_data

class LaikagoEnv(gym.Env):
    def __init__(self):
        super(LaikagoEnv, self).__init__()

        # PyBulletの物理エンジンをDIRECTモードで起動
        self.physicsClient = p.connect(p.DIRECT)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.8)

        self.laikago_path = "laikago/laikago_toes.urdf"
        self.num_joints = 12  # Laikagoのモーター数
        self.dt = 1. / 240

        # 観測空間：関節角度と速度のセット
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf,
                                            shape=(self.num_joints * 2,),
                                            dtype=np.float32)

        # 行動空間：各関節角度の変化量（[-0.5,0.5]）
        self.action_space = spaces.Box(low=-0.5, high=0.5,
                                       shape=(self.num_joints,), dtype=np.float32)

        self.reset()
        self.motor_ids = [0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14]
        self.num_joints = len(self.motor_ids)

    def reset(self):
        # シミュレーションリセット
        p.resetSimulation()
        p.setGravity(0, 0, -9.8)
        p.loadURDF("plane.urdf")

        # ロボット読み込み（ベースは少し高く）
        self.robot = p.loadURDF(
            self.laikago_path,
            basePosition=[0, 0, 0.45],
            baseOrientation=p.getQuaternionFromEuler([np.pi/2, 0, -np.pi/2])
        )
        self.step_counter = 0

        # 初期姿勢（立位に近い角度）を設定
        # 4脚分 hip, thigh, calf の順で12関節分
        initial_angles = {
                0: 0.25,    # FR ヒップ横軸 (右前足外側に開く)
                1: 0.6,     # FR ヒップ縦軸
                2: -1.1,    # FR 膝

                4: -0.25,   # FL ヒップ横軸 (左前足外側に開く、符号逆)
                5: 0.6,     # FL ヒップ縦軸
                6: -1.1,    # FL 膝

                8: 0.25,    # RR ヒップ横軸 (右後ろ足外側に開く)
                9: 0.6,     # RR ヒップ縦軸
                10: -1.1,   # RR 膝

                12: -0.25,  # RL ヒップ横軸 (左後ろ足外側に開く)
                13: 0.6,    # RL ヒップ縦軸
                14: -1.1,   # RL 膝
            }

        for joint_id, angle in initial_angles.items():
            p.resetJointState(self.robot, joint_id, angle)
            #p.setJointMotorControl2(self.robot, joint_id,
                                   # controlMode=p.POSITION_CONTROL,
                                   # targetPosition=angle,
                                   # force=30)
            p.setJointMotorControl2(self.robot, joint_id,
                            controlMode=p.POSITION_CONTROL,
                            force=30)

        for _ in range(100):
            p.stepSimulation()

        return self._get_obs()

    def _get_obs(self):
        # 関節角度・速度の取得
        joint_states = [p.getJointState(self.robot, i) for i in range(self.num_joints)]
        joint_pos = np.array([x[0] for x in joint_states])
        joint_vel = np.array([x[1] for x in joint_states])
        return np.concatenate([joint_pos, joint_vel])

    def step(self, action):
        # PD制御パラメータ
        for i in range(self.num_joints):
            p.setJointMotorControl2(
                bodyIndex=self.robot,
                jointIndex=i,
                controlMode=p.POSITION_CONTROL,
                targetPosition=action[i],
                force=20
            )

        # 4ステップ分シミュレーション進める
        for _ in range(10):
            p.stepSimulation()

        self.step_counter += 1
        obs = self._get_obs()

        # 報酬計算（前進速度 - 高さ維持のペナルティ）
        pos, _ = p.getBasePositionAndOrientation(self.robot)
        vel, _ = p.getBaseVelocity(self.robot)
        height_penalty = abs(pos[2] - 0.48)
        forward_reward = max(vel[1], 0)
        reward = forward_reward - 1.0 * height_penalty

        if pos[2] < 0.2:
            reward = -20
            done = True

        done = self.step_counter > 1000
        return obs, reward, done, {}

    def render(self, mode='human'):
        pass

    def close(self):
        p.disconnect()

In [6]:
!pip install 'shimmy>=2.0'

In [10]:
from stable_baselines3 import PPO

env = LaikagoEnv()
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=2000000)

ストリーミング出力は最後の 5000 行に切り捨てられました。
|    loss                 | 1.01e+04    |
|    n_updates            | 2610        |
|    policy_gradient_loss | -0.0284     |
|    std                  | 0.67        |
|    value_loss           | 2.92e+04    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -1.51e+04   |
| time/                   |             |
|    fps                  | 181         |
|    iterations           | 263         |
|    time_elapsed         | 2971        |
|    total_timesteps      | 538624      |
| train/                  |             |
|    approx_kl            | 0.013109096 |
|    clip_fraction        | 0.147       |
|    clip_range           | 0.2         |
|    entropy_loss         | -12.2       |
|    explained_variance   | 0.229       |
|    learning_rate        | 0.0003      |
|    loss                 | 2.3e+04     |
| 

In [14]:
obs = env.reset()
frames = []
#1860
for _ in range(1860):
    action, _ = model.predict(obs)
    obs, _, done, _ = env.step(action)

    # カメラ画像取得（視点補正あり）
    _, _, rgb, _, _ = p.getCameraImage(
        320, 240,
        viewMatrix=p.computeViewMatrixFromYawPitchRoll(
            cameraTargetPosition=[0, 0, 0.3],
            distance=1.5,
            yaw=90, pitch=-30, roll=0, upAxisIndex=2
        ),
        projectionMatrix=p.computeProjectionMatrixFOV(fov=60, aspect=1.0, nearVal=0.1, farVal=10.0)
    )
    img = np.reshape(rgb, (240, 320, 4))[:, :, :3]
    frames.append(img)

# 保存
import imageio
imageio.mimsave("laikago_learned.mp4", frames, fps=30)